In [6]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
import joblib

CNN = 0
BREITBART = 1
NYT = 2

articles = pd.read_csv('articles1.csv')
#creates dataframe with only CNN, and one with only Breitbart
cnn = articles.loc[articles['publication'] == 'CNN'][['title', 'publication']]
breitbart = articles.loc[articles['publication'] == 'Breitbart'][['title', 'publication']]
nyt = articles.loc[articles['publication'] == 'New York Times'][['title', 'publication']]
#sets publication column values to integers based on publication
cnn['publication'] = CNN
breitbart['publication'] = BREITBART
nyt['publication'] = NYT
#combines left and right leaning
articles = cnn.append([breitbart, nyt])
#removes self-identification
articles['title'] = articles['title'].str.replace('CNN', '')
articles['title'] = articles['title'].str.replace('Breitbart', '')
articles['title'] = articles['title'].str.replace('New York Times', '')
#randomizes rows
articles = articles.sample(frac=1)

stemmer = SnowballStemmer('english')
words = stopwords.words("english")

articles['cleaned'] = articles['title'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

X_train, X_test, y_train, y_test = train_test_split(articles['cleaned'], articles['publication'], test_size=0.2)

pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english")),
                     ('chi',  SelectKBest(k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l2', max_iter=1000, dual=False))])

model = pipeline.fit(X_train, y_train)

joblib.dump(model, 'demo_model.joblib')

vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)

target_names = ['CNN', 'Breitbart', 'New York Times']

print("\nAccuracy: " + str(round(model.score(X_test, y_test)*100, 2)) + "%\n")

print("\nTop 10 keywords per publication:\n")
for i, label in enumerate(target_names):
    top10 = np.argsort(clf.coef_[i])[-10:]
    print("%s: %s" % (label, " ".join(feature_names[top10])))





Accuracy: 69.22%


Top 10 keywords per publication:

CNN: nepal simpson fraud claim isi unc travel ban realiti check student news fast fact opinion
Breitbart: wapo delingpol cruz msnbc islam dure migrant exclus milo hillari
New York Times: campaign presid fargo know don assail broadway inquiri brief brooklyn hillari clinton california today
